# Tutorial: Creating Language Agents to interact with Aviary environments

---

This tutorial will guide you in creating language agents that interact with [Aviary](https://github.com/Future-House/aviary) environments. We'll start by introducing a few basic concepts from the Aviary and [LDP](https://github.com/Future-House/ldp) frameworks, then show how to define agents that can engage with existing environments. First, we’ll build a simple agent, then extend it to create one that follows system prompts for guided interactions.

---

## **Pre-requisites**

You’ll need to clone and install the *aviary* and *ldp* repositories. Execute the cell below to so so


In [9]:
# Install the aviary repository with gsm8k and hotpotqa envs
!pip install "fhaviary[gsm8k,hotpotqa]"

# Clone the ldp repository
!git clone https://github.com/Future-House/ldp.git

# Navigate into the ldp directory and install it
!cd ldp && pip install -e .

fatal: destination path 'ldp' already exists and is not an empty directory.
Obtaining file:///Users/albertbou/repos/ldp/docs/ldp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ldp (pyproject.toml) ... done
  Created wheel for ldp: filename=ldp-0.11.4.dev4+g7c0b2fe-0.editable-py3-none-any.whl size=12686 sha256=8f4df82963e52a7d5501d00172cba7f157fff89191ef6bc9bf14f307f538c5d9
  Stored in directory: /private/var/folders/n0/8h147sh56r502191rhk945nr0000gn/T/pip-ephem-wheel-cache-1cyzlo34/wheels/18/9c/0c/269e998ad37101d2793c62db212888e4b77ae7fe44ed9283b2
Successfully built ldp
  Attempting uninstall: ldp
    Found existing installation: ldp 0.11.4.dev4+g7c0b2fe
    Uninstalling ldp-0.11.4.dev4+g7c0b2fe:
      Successfully uninstalled ldp-0.11.4.dev4+g7c0b2fe


You will need to set the OPENAI_API_KEY and ANTHROPIC_API_KEY

In [2]:
import os

# Set the environment variables
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key"

# 1. Background

[Aviary](https://github.com/Future-House/aviary) is our framework supporting diverse language environments, where actions are tools available to agents. [LDP](https://github.com/Future-House/ldp) is our framework for creating and training language agents.

Below, we briefly define some key classes and concepts from these libraries for context:

**From Aviary**  
- **Message**: Used by language agents and environments for communication.
- **Tool**: Defines an environmental tool that an agent can use to accomplish its task. Different environments offer different tools.
- **ToolRequestMessage**, **ToolResponseMessage**: Specialized `Message` subclasses for tool requests and responses.

**From LDP**  
- **Agent**: An entity that interacts with the environment, mapping observations to tool request actions.
- **Compute Graph**: A network of operations representing data flow through an agent, storing information and parameters useful for learning. Similar to compute graphs in libraries like TensorFlow or PyTorch, LDP handles this internally, so in-depth knowledge isn’t necessary for this tutorial. However, some methods will have a `@compute_graph` decorator due to this structure.
- **Op**: Represents an operation within the agent. LDP includes various operations (Ops), such as API LLM calls, API embedding calls, or PyTorch module handling. These operations form the compute graph.
- **OpResult**: A class representing the output of an `Op`.


## Section 2: Defining a Simple Agent¶

We will start by defining the simplest, most minimal agent possible.

### 2.1 Defining an Agent’s State

In typical reinforcement learning (RL), an agent receives an observation and generates an action while maintaining an internal state. However, as a design choice here, we make the agents independent of the state and provide them with the current state at each step. This means that the agent will receive an “agent state” containing all relevant information needed to make the next decision at any given step, instead of keeping it internally. This approach enables us, for example, to use a single agent instance to process multiple rollouts in parallel.

The state includes all relevant information about a rollout in progress and required by the aget to make the next decision. In the simplest case, this comprises the current list of messages from the start of the episode and the tools available to the agent. We define such a state as follows:


In [3]:
from aviary.core import Message, Tool, ToolRequestMessage
from pydantic import BaseModel, Field

from ldp.agent import Agent
from ldp.graph import OpResult


class MySimpleAgentState(BaseModel):
    """Simple bucket to store available tools and previous messages."""

    tools: list[Tool] = Field(default_factory=list)
    messages: list[Message] = Field(default_factory=list)

    def get_next_state(
        self,
        obs: list[Message] | None = None,
    ):
        """
        Return the next agent state based on the current state and optional observation messages.

        Args:
            obs: Optional observation messages to use in creating the next state.

        Returns:
            The next agent state.
        """
        return type(self)(
            tools=self.tools,
            messages=self.messages + (obs or []),
        )

### 2.2 Defining Agent 

The `MySimpleAgent` class is a basic agent that utilizes a language model (LLM) to decide which tools to invoke based on observations from its environment. It manages a `MySimpleAgentState` to store relevant information. The agent's core functionality involves querying an LLM to determine the next tool request and updating its state accordingly.

All agents in our framework must implement two asynchronous methods:

1. **`init_state`**: Initializes the agent’s internal state by accepting a list of tools the agent can use (provided by the environment).
2. **`get_asv`**: Processes observations to update the agent's state and decide which tool to use next. It returns the tool request, the updated agent state, and a value. This value indicates the quality of the current state, which may benefit certain learning algorithms. In our case, we set it to 0.0.

This setup enables `MySimpleAgent` to iteratively assess and respond to new information, invoking the appropriate tool with each decision step.


In [4]:
from typing import Any

from ldp.graph import LLMCallOp, compute_graph


class MySimpleAgent(BaseModel, Agent[MySimpleAgentState]):
    """Simple agent that can invoke tools with a language model."""

    llm_model: dict[str, Any] = Field(
        default={
            "model": "gpt-4o-2024-08-06",
            "temperature": 0.1,
        },  # This model is cheap, fast, and decent
        description="Configuration for the LLM model.",
    )

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Create a Op that calls an LLM
        self._llm_call_op = LLMCallOp()

    async def init_state(self, tools: list[Tool]) -> MySimpleAgentState:
        return MySimpleAgentState(tools=tools)

    @compute_graph()
    async def get_asv(
        self, agent_state: MySimpleAgentState, obs: list[Message]
    ) -> tuple[OpResult[ToolRequestMessage], MySimpleAgentState, float]:
        # Obtain the next agent state, given the environment observation
        next_state = agent_state.get_next_state(obs)

        # Call agent language model to ask for the next tool to use
        result = await self._llm_call_op(
            self.llm_model, msgs=next_state.messages, tools=next_state.tools
        )

        # Extend the the agent state with the new ToolRequestMessage
        next_state.messages = [*next_state.messages, result.value]

        # Agent returns an OpResult, the next agent state and the value, which we set to 0.0
        return result, next_state, 0.0

### 2.3 Testing Agent

Let's define an environment from aviary and ensure our agent can interact with it. The GSM8K environment presents simple mathematical problems to the agent and includes both a calculator tool and a tool for submitting the final answer. The agent receives a reward only after submitting a correct answer.

In [6]:
import asyncio

from gsm8k.env import GSM8kDataset


async def main(idx=0):
    env = GSM8kDataset(split="train").get_new_env_by_idx(idx)
    agent = MySimpleAgent()

    # Get initial question, available tools from the environment
    obs, tools = await env.reset()
    print(f"Question: {obs[0].content}")

    # Get initial agent state
    agent_state = await agent.init_state(tools=tools)

    step = 1
    done = False
    while not done:
        action, agent_state, _ = await agent.get_asv(agent_state, obs)
        obs, reward, done, _ = await env.step(action.value)
        print(f"Agent step {step} - {print_action_obs(action, obs)}, reward {reward}")
        step += 1

    print("Finished! \n")


def print_action_obs(action: ToolRequestMessage, obs: list):
    tool_calls = action.value.tool_calls
    msg = ""
    for tool_call, tool_answer in zip(tool_calls, obs, strict=True):
        tool_name = tool_call.function.name
        tool_args = tool_call.function.arguments
        msg += f"{tool_name}({tool_args}), answer: {tool_answer.content} "
    return msg

async def run_all():
    await asyncio.gather(*(main(i) for i in range(3)))

for i in range(3):
    asyncio.run(main(i))

RuntimeError: asyncio.run() cannot be called from a running event loop

## Section 3: Specific Prompt Agent

Let's extend our agent a bit further. It is common practice to provide system-level guidelines to an agent to help guide its behavior and improve its responses. These textual guidelines provide context or rules that help the agent interpret its environment, make better decisions, and align with specific objectives. We’ll incorporate these guidelines to enhance our agent’s performance and ensure it consistently follows our intended framework.

### 3.1 Defining Agent 

The changes are minimal with respect to our `MySimpleAgent` class, and we can reuse the `MySimpleAgentState`.

In [7]:
from typing import Any

from ldp.graph import compute_graph


class MyGuidedAgent(BaseModel, Agent[MySimpleAgentState]):
    """Simple agent that can invoke tools with a language model."""

    llm_model: dict[str, Any] = Field(
        default={
            "model": "gpt-4o-2024-08-06",
            "temperature": 0.1,
        },  # This model is cheap, fast, and decent
        description="Configuration for the LLM model.",
    )

    guidelines_msg: Message = Field(
        default=Message(role="system", content=""),
        description="Initial guidelines to be shown to the LLM.",
    )

    def __init__(self, guidelines: str = "", **kwargs):
        super().__init__(**kwargs)

        self.guidelines_msg = Message(role="system", content=guidelines)

        # Create a Op that calls an LLM
        self._llm_call_op = LLMCallOp()

    async def init_state(self, tools: list[Tool]) -> MySimpleAgentState:
        return MySimpleAgentState(tools=tools)

    @compute_graph()
    async def get_asv(
        self, agent_state: MySimpleAgentState, obs: list[Message]
    ) -> tuple[OpResult[ToolRequestMessage], MySimpleAgentState, float]:
        # Obtain the next agent state, given the environment observation
        next_state = agent_state.get_next_state(obs)

        # Call agent language model to ask for the next tool to use
        result = await self._llm_call_op(
            self.llm_model,
            msgs=[
                self.guidelines_msg,
                *next_state.messages,
            ],  # We prepend the system guidelines here!
            tools=next_state.tools,
        )

        # Extend the the agent state with the new ToolRequestMessage
        next_state.messages = [*next_state.messages, result.value]

        # Agent returns an OpResult, the next agent state and the value, which we set to 0.0
        return result, next_state, 0.0

### 3.2 Testing Agent

An simple example where our improved agent can be useful is the HotPotQA environment. The HotPotQA is a question-answering environment. However, the environment only considers the answer correct if it exactly matches the ground truth. Most agents tend to respond with a full sentence that, although containing the correct answer, is marked as incorrect. By providing specific guidelines, we can guide our agent to answer with only the precise information requested. Let's test this by running the environment first without guidelines, and then with them.

In [10]:
from hotpotqa import HotPotQADataset


async def main(idx=0, guidelines=""):
    env = HotPotQADataset(split="train").get_new_env_by_idx(idx)
    agent = MyGuidedAgent(guidelines=guidelines)

    # Get initial question, available tools from the environment
    obs, tools = await env.reset()
    print(f"{obs[0].content}")

    # Get initial agent state
    agent_state = await agent.init_state(tools=tools)

    step = 1
    done = False
    while not done:
        action, agent_state, _ = await agent.get_asv(agent_state, obs)
        obs, reward, done, _ = await env.step(action.value)
        print(f"Agent step {step} - {print_action_obs(action, obs)}, reward {reward}")
        step += 1

    print("Finished! \n")


for i in range(1):
    asyncio.run(main(i))

guidelines = (
    "Answer the questions. "
    "Return only the specific information asked. "
    "If asked for a name, only the name; if asked for a year, only the year, etc."
)

for i in range(1):
    asyncio.run(main(i, guidelines))

ModuleNotFoundError: No module named 'hotpotqa'

## Section 3: Advanced Agent Definition

Many more options are possible. New agents can be created and extended by modifying their `get_asv` method, and if additional information is needed, their state can also be adjusted. Ideas include agents that reflect before taking action, receive feedback from an oracle, or simulate multiple scenarios before making a decision.